In [1]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Configurations
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Loading the data

In [2]:
# Import xlsx dataset into pandas dataframe
data = pd.read_excel('/Users/diogocarvalho/Documents/GitHub/Case-2--Siemens-Sales-Forecast/Data/Case2_Market data.xlsx', header=[0,1,2])

In [3]:
# Verify how the data looks
data.head()

Unnamed: 0_level_0  \
  Index 2010=100 (if not otherwise noted)   
                                     date   
0                                  2004m2   
1                                  2004m3   
2                                  2004m4   
3                                  2004m5   
4                                  2004m6   

                                     China  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO156   
0                                16.940704   
1                                23.711852   
2                                24.435235   
3                                23.708115   
4                                27.009138   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP156   
0                               16.940704   
1                               23.711852   
2                               24.435235   
3                               23.708115   
4                               27.009138   

                                    France  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO250   
0                               112.091273   
1                               136.327976   
2                               117.791806   
3                               109.002541   
4                               133.785737   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP250   
0                               83.458866   
1                              106.168192   
2                               92.007646   
3                               85.696486   
4                              106.641482   

                                   Germany  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO276   
0                                82.623037   
1                               100.556582   
2                                89.653203   
3                                86.880571   
4                                99.010814   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP276   
0                               79.452532   
1                               97.012918   
2                               84.932358   
3                               82.372794   
4                               95.108740   

                                     Italy  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO380   
0                               124.289603   
1                               143.411662   
2                               129.083828   
3                               135.590391   
4                               136.424935   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP380   
0                               86.560493   
1                              106.344544   
2                               95.579673   
3                              100.087039   
4                              110.889719   

                                     Japan  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO392   
0                               109.334010   
1                               140.884616   
2                               105.853579   
3                               101.864777   
4                               120.332920   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP392   
0                              110.495272   
1                              144.686166   
2                              102.655769   
3                              100.305285   
4                              119.616380   

            

In [4]:
# Check the shape of the data
data.shape

(219, 48)

In [5]:
# Check the columns of the data
data.columns

MultiIndex([('Unnamed: 0_level_0', ...),
            (             'China', ...),
            (             'China', ...),
            (            'France', ...),
            (            'France', ...),
            (           'Germany', ...),
            (           'Germany', ...),
            (             'Italy', ...),
            (             'Italy', ...),
            (             'Japan', ...),
            (             'Japan', ...),
            (       'Switzerland', ...),
            (       'Switzerland', ...),
            (    'United Kingdom', ...),
            (    'United Kingdom', ...),
            (     'United States', ...),
            (     'United States', ...),
            (            'Europe', ...),
            (            'Europe', ...),
            (            'Europe', ...),
            (            'Europe', ...),
            (            'Europe', ...),
            (            'Europe', ...),
            (            'Europe', ...),
            (   

In [6]:
# Join the header levels
data.columns = [
    '_'.join([str(level) for level in col_tuple]).strip()
    for col_tuple in data.columns
]


In [7]:
data.head()

,Unnamed: 0_level_0_Index 2010=100 (if not otherwise noted)_date,China_Production Index Machinery & Electricals_MAB_ELE_PRO156,China_Shipments Index Machinery & Electricals_MAB_ELE_SHP156,France_Production Index Machinery & Electricals_MAB_ELE_PRO250,France_Shipments Index Machinery & Electricals_MAB_ELE_SHP250,Germany_Production Index Machinery & Electricals_MAB_ELE_PRO276,Germany_Shipments Index Machinery & Electricals_MAB_ELE_SHP276,Italy_Production Index Machinery & Electricals_MAB_ELE_PRO380,Italy_Shipments Index Machinery & Electricals_MAB_ELE_SHP380,Japan_Production Index Machinery & Electricals_MAB_ELE_PRO392,Japan_Shipments Index Machinery & Electricals_MAB_ELE_SHP392,Switzerland_Production Index Machinery & Electricals_MAB_ELE_PRO756,Switzerland_Shipments Index Machinery & Electricals_MAB_ELE_SHP756,United Kingdom_Production Index Machinery & Electricals_MAB_ELE_PRO826,United Kingdom_Shipments Index Machinery & Electricals_MAB_ELE_SHP826,United States_Production Index Machinery & Electricals_MAB_ELE_PRO840,United States_Shipments Index Machinery & Electricals_MAB_ELE_SHP840,Europe_Production Index Machinery & Electricals_MAB_ELE_PRO1100,Europe_Shipments Index Machinery & Electricals_MAB_ELE_SHP1100,Europe_World: Price of Base Metals_RohiBASEMET1000_org,Europe_World: Price of Energy_RohiENERGY1000_org,Europe_World: Price of Metals & Minerals_RohiMETMIN1000_org,Europe_World: Price of Natural gas index_RohiNATGAS1000_org,"Europe_World: Price of Crude oil, average_RohCRUDE_PETRO1000_org",Europe_World: Price of Copper_RohCOPPER1000_org,Europe_United States: EUR in LCU_WKLWEUR840_org,Producer Prices_United States: Electrical equipment_PRI27840_org,Producer Prices_United Kingdom: Electrical equipment_PRI27826_org,Producer Prices_Italy: Electrical equipment_PRI27380_org,Producer Prices_France: Electrical equipment_PRI27250_org,Producer Prices_Germany: Electrical equipment_PRI27276_org,Producer Prices_China: Electrical equipment_PRI27156_org,production index_United States: Machinery and equipment n.e.c._PRO28840_org,production index_World: Machinery and equipment n.e.c._PRO281000_org,production index_Switzerland: Machinery and equipment n.e.c._PRO28756_org,production index_United Kingdom: Machinery and equipment n.e.c._PRO28826_org,production index_Italy: Machinery and equipment n.e.c._PRO28380_org,production index_Japan: Machinery and equipment n.e.c._PRO28392_org,production index_France: Machinery and equipment n.e.c._PRO28250_org,production index_Germany: Machinery and equipment n.e.c._PRO28276_org,production index_United States: Electrical equipment_PRO27840_org,production index_World: Electrical equipment_PRO271000_org,production index_Switzerland: Electrical equipment_PRO27756_org,production index_United Kingdom: Electrical equipment_PRO27826_org,production index_Italy: Electrical equipment_PRO27380_org,production index_Japan: Electrical equipment_PRO27392_org,production index_France: Electrical equipment_PRO27250_org,production index_Germany: Electrical equipment_PRO27276_org
0,2004m2,16.940704,16.940704,112.091273,83.458866,82.623037,79.452532,124.289603,86.560493,109.334010,110.495272,91.221862,89.987275,111.353812,73.601265,107.601400,79.240230,97.122911,80.098530,54.039811,44.123338,48.747945,87.076974,39.639458,36.623832,1.2646,78.969864,80.757423,93.020027,NaN,93.230453,NaN,102.491722,97.597374,97.100000,106.191977,116.790276,110.890034,118.274109,80.829010,117.723991,NaN,81.100000,120.706516,141.510864,106.161262,102.077057,85.913200
1,2004m3,23.711852,23.711852,136.327976,106.168192,100.556582,97.012918,143.411662,106.344544,140.884616,144.686166,85.866287,79.883583,127.558608,84.047595,110.187364,98.619024,113.783904,96.015929,54.666162,47.588957,49.256157,87.192705,42.592034,39.931055,1.2262,79.673569,80.962135,93.540268,NaN,93.335678,NaN,105.627480,113.224892,91.195116,121.625075,139.288391,141.176853,148.121841,102.130104,119.220779,NaN,76.690307,138.309550,152.880234,140.288741,117.225685,97.670815
2,2004m

In [8]:
data.columns

Index(['Unnamed: 0_level_0_Index 2010=100 (if not otherwise noted)_date',
       'China_Production Index Machinery & Electricals_MAB_ELE_PRO156',
       'China_Shipments Index Machinery & Electricals_MAB_ELE_SHP156',
       'France_Production Index Machinery & Electricals_MAB_ELE_PRO250',
       'France_Shipments Index Machinery & Electricals_MAB_ELE_SHP250',
       'Germany_Production Index Machinery & Electricals_MAB_ELE_PRO276',
       'Germany_Shipments Index Machinery & Electricals_MAB_ELE_SHP276',
       'Italy_Production Index Machinery & Electricals_MAB_ELE_PRO380',
       'Italy_Shipments Index Machinery & Electricals_MAB_ELE_SHP380',
       'Japan_Production Index Machinery & Electricals_MAB_ELE_PRO392',
       'Japan_Shipments Index Machinery & Electricals_MAB_ELE_SHP392',
       'Switzerland_Production Index Machinery & Electricals_MAB_ELE_PRO756',
       'Switzerland_Shipments Index Machinery & Electricals_MAB_ELE_SHP756',
       'United Kingdom_Production Index Machinery 

In [9]:
# change first columne name to "Date"
data.rename(columns={'Unnamed: 0_level_0_Index 2010=100 (if not otherwise noted)_date': 'Date'}, inplace=True)

In [10]:
# Verify changes
data.head()

,Date,China_Production Index Machinery & Electricals_MAB_ELE_PRO156,China_Shipments Index Machinery & Electricals_MAB_ELE_SHP156,France_Production Index Machinery & Electricals_MAB_ELE_PRO250,France_Shipments Index Machinery & Electricals_MAB_ELE_SHP250,Germany_Production Index Machinery & Electricals_MAB_ELE_PRO276,Germany_Shipments Index Machinery & Electricals_MAB_ELE_SHP276,Italy_Production Index Machinery & Electricals_MAB_ELE_PRO380,Italy_Shipments Index Machinery & Electricals_MAB_ELE_SHP380,Japan_Production Index Machinery & Electricals_MAB_ELE_PRO392,Japan_Shipments Index Machinery & Electricals_MAB_ELE_SHP392,Switzerland_Production Index Machinery & Electricals_MAB_ELE_PRO756,Switzerland_Shipments Index Machinery & Electricals_MAB_ELE_SHP756,United Kingdom_Production Index Machinery & Electricals_MAB_ELE_PRO826,United Kingdom_Shipments Index Machinery & Electricals_MAB_ELE_SHP826,United States_Production Index Machinery & Electricals_MAB_ELE_PRO840,United States_Shipments Index Machinery & Electricals_MAB_ELE_SHP840,Europe_Production Index Machinery & Electricals_MAB_ELE_PRO1100,Europe_Shipments Index Machinery & Electricals_MAB_ELE_SHP1100,Europe_World: Price of Base Metals_RohiBASEMET1000_org,Europe_World: Price of Energy_RohiENERGY1000_org,Europe_World: Price of Metals & Minerals_RohiMETMIN1000_org,Europe_World: Price of Natural gas index_RohiNATGAS1000_org,"Europe_World: Price of Crude oil, average_RohCRUDE_PETRO1000_org",Europe_World: Price of Copper_RohCOPPER1000_org,Europe_United States: EUR in LCU_WKLWEUR840_org,Producer Prices_United States: Electrical equipment_PRI27840_org,Producer Prices_United Kingdom: Electrical equipment_PRI27826_org,Producer Prices_Italy: Electrical equipment_PRI27380_org,Producer Prices_France: Electrical equipment_PRI27250_org,Producer Prices_Germany: Electrical equipment_PRI27276_org,Producer Prices_China: Electrical equipment_PRI27156_org,production index_United States: Machinery and equipment n.e.c._PRO28840_org,production index_World: Machinery and equipment n.e.c._PRO281000_org,production index_Switzerland: Machinery and equipment n.e.c._PRO28756_org,production index_United Kingdom: Machinery and equipment n.e.c._PRO28826_org,production index_Italy: Machinery and equipment n.e.c._PRO28380_org,production index_Japan: Machinery and equipment n.e.c._PRO28392_org,production index_France: Machinery and equipment n.e.c._PRO28250_org,production index_Germany: Machinery and equipment n.e.c._PRO28276_org,production index_United States: Electrical equipment_PRO27840_org,production index_World: Electrical equipment_PRO271000_org,production index_Switzerland: Electrical equipment_PRO27756_org,production index_United Kingdom: Electrical equipment_PRO27826_org,production index_Italy: Electrical equipment_PRO27380_org,production index_Japan: Electrical equipment_PRO27392_org,production index_France: Electrical equipment_PRO27250_org,production index_Germany: Electrical equipment_PRO27276_org
0,2004m2,16.940704,16.940704,112.091273,83.458866,82.623037,79.452532,124.289603,86.560493,109.334010,110.495272,91.221862,89.987275,111.353812,73.601265,107.601400,79.240230,97.122911,80.098530,54.039811,44.123338,48.747945,87.076974,39.639458,36.623832,1.2646,78.969864,80.757423,93.020027,NaN,93.230453,NaN,102.491722,97.597374,97.100000,106.191977,116.790276,110.890034,118.274109,80.829010,117.723991,NaN,81.100000,120.706516,141.510864,106.161262,102.077057,85.913200
1,2004m3,23.711852,23.711852,136.327976,106.168192,100.556582,97.012918,143.411662,106.344544,140.884616,144.686166,85.866287,79.883583,127.558608,84.047595,110.187364,98.619024,113.783904,96.015929,54.666162,47.588957,49.256157,87.192705,42.592034,39.931055,1.2262,79.673569,80.962135,93.540268,NaN,93.335678,NaN,105.627480,113.224892,91.195116,121.625075,139.288391,141.176853,148.121841,102.130104,119.220779,NaN,76.690307,138.309550,152.880234,140.288741,117.225685,97.670815
2,2004m4,24.435235,24.435235,117.791806,92.007646,89.653203,84.932

In [11]:
# Make Date the index
data.set_index('Date', inplace=True)

In [22]:
# Verify changes
data.head()

,China_Production Index Machinery & Electricals_MAB_ELE_PRO156,China_Shipments Index Machinery & Electricals_MAB_ELE_SHP156,France_Production Index Machinery & Electricals_MAB_ELE_PRO250,France_Shipments Index Machinery & Electricals_MAB_ELE_SHP250,Germany_Production Index Machinery & Electricals_MAB_ELE_PRO276,Germany_Shipments Index Machinery & Electricals_MAB_ELE_SHP276,Italy_Production Index Machinery & Electricals_MAB_ELE_PRO380,Italy_Shipments Index Machinery & Electricals_MAB_ELE_SHP380,Japan_Production Index Machinery & Electricals_MAB_ELE_PRO392,Japan_Shipments Index Machinery & Electricals_MAB_ELE_SHP392,Switzerland_Production Index Machinery & Electricals_MAB_ELE_PRO756,Switzerland_Shipments Index Machinery & Electricals_MAB_ELE_SHP756,United Kingdom_Production Index Machinery & Electricals_MAB_ELE_PRO826,United Kingdom_Shipments Index Machinery & Electricals_MAB_ELE_SHP826,United States_Production Index Machinery & Electricals_MAB_ELE_PRO840,United States_Shipments Index Machinery & Electricals_MAB_ELE_SHP840,Europe_Production Index Machinery & Electricals_MAB_ELE_PRO1100,Europe_Shipments Index Machinery & Electricals_MAB_ELE_SHP1100,Europe_World: Price of Base Metals_RohiBASEMET1000_org,Europe_World: Price of Energy_RohiENERGY1000_org,Europe_World: Price of Metals & Minerals_RohiMETMIN1000_org,Europe_World: Price of Natural gas index_RohiNATGAS1000_org,"Europe_World: Price of Crude oil, average_RohCRUDE_PETRO1000_org",Europe_World: Price of Copper_RohCOPPER1000_org,Europe_United States: EUR in LCU_WKLWEUR840_org,Producer Prices_United States: Electrical equipment_PRI27840_org,Producer Prices_United Kingdom: Electrical equipment_PRI27826_org,Producer Prices_Italy: Electrical equipment_PRI27380_org,Producer Prices_France: Electrical equipment_PRI27250_org,Producer Prices_Germany: Electrical equipment_PRI27276_org,Producer Prices_China: Electrical equipment_PRI27156_org,production index_United States: Machinery and equipment n.e.c._PRO28840_org,production index_World: Machinery and equipment n.e.c._PRO281000_org,production index_Switzerland: Machinery and equipment n.e.c._PRO28756_org,production index_United Kingdom: Machinery and equipment n.e.c._PRO28826_org,production index_Italy: Machinery and equipment n.e.c._PRO28380_org,production index_Japan: Machinery and equipment n.e.c._PRO28392_org,production index_France: Machinery and equipment n.e.c._PRO28250_org,production index_Germany: Machinery and equipment n.e.c._PRO28276_org,production index_United States: Electrical equipment_PRO27840_org,production index_World: Electrical equipment_PRO271000_org,production index_Switzerland: Electrical equipment_PRO27756_org,production index_United Kingdom: Electrical equipment_PRO27826_org,production index_Italy: Electrical equipment_PRO27380_org,production index_Japan: Electrical equipment_PRO27392_org,production index_France: Electrical equipment_PRO27250_org,production index_Germany: Electrical equipment_PRO27276_org
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004m2,16.940704,16.940704,112.091273,83.458866,82.623037,79.452532,124.289603,86.560493,109.334010,110.495272,91.221862,89.987275,111.353812,73.601265,107.601400,79.240230,97.122911,80.098530,54.039811,44.123338,48.747945,87.076974,39.639458,36.623832,1.2646,78.969864,80.757423,93.020027,NaN,93.230453,NaN,102.491722,97.597374,97.100000,106.191977,116.790276,110.890034,118.274109,80.829010,117.723991,NaN,81.100000,120.706516,141.510864,106.161262,102.077057,85.913200
2004m3,23.711852,23.711852,136.327976,106.168192,100.556582,97.012918,143.411662,106.344544,140.884616,144.686166,85.866287,79.883583,127.558608,84.047595,110.187364,98.619024,113.783904,96.015929,54.666162,47.588957,49.256157,87.192705,42.592034,39.931055,1.2262,79.673569,80.962135,93.540268,NaN,93.335678,NaN,105.627480,113.224892,91.195116,121.625075,139.288391,141.176853,148.121841,102.130104,119.220779,NaN,76.690307,138.309550,152.880234,140.288741,117.225685,97.670815
2004m4,24.435235,24.435

Columns from index 0-18 correspond to sales data (or costs?) <br>
Columns from index 18-24 correspond to price of materials, so keep them in separate dataframe <br>
Columns from index 25-30 corresponde to producer prices <br>
Columns from index 30 onwards correspond to production index

In [26]:
# what is the index of the column 'Europe_World: Price of Base Metals_RohiBASEMET1000_org'
data.columns.get_loc('Producer Prices_China: Electrical equipment_PRI27156_org')

30

In [13]:

data.columns

Index(['China_Production Index Machinery & Electricals_MAB_ELE_PRO156',
       'China_Shipments Index Machinery & Electricals_MAB_ELE_SHP156',
       'France_Production Index Machinery & Electricals_MAB_ELE_PRO250',
       'France_Shipments Index Machinery & Electricals_MAB_ELE_SHP250',
       'Germany_Production Index Machinery & Electricals_MAB_ELE_PRO276',
       'Germany_Shipments Index Machinery & Electricals_MAB_ELE_SHP276',
       'Italy_Production Index Machinery & Electricals_MAB_ELE_PRO380',
       'Italy_Shipments Index Machinery & Electricals_MAB_ELE_SHP380',
       'Japan_Production Index Machinery & Electricals_MAB_ELE_PRO392',
       'Japan_Shipments Index Machinery & Electricals_MAB_ELE_SHP392',
       'Switzerland_Production Index Machinery & Electricals_MAB_ELE_PRO756',
       'Switzerland_Shipments Index Machinery & Electricals_MAB_ELE_SHP756',
       'United Kingdom_Production Index Machinery & Electricals_MAB_ELE_PRO826',
       'United Kingdom_Shipments Index Mach

## sales dataframe

In [14]:
# Slice dataframe and keep the first 20 columns
sales = data.iloc[:, :18]

In [15]:
sales.shape

(219, 18)

In [16]:
sales.columns

Index(['China_Production Index Machinery & Electricals_MAB_ELE_PRO156',
       'China_Shipments Index Machinery & Electricals_MAB_ELE_SHP156',
       'France_Production Index Machinery & Electricals_MAB_ELE_PRO250',
       'France_Shipments Index Machinery & Electricals_MAB_ELE_SHP250',
       'Germany_Production Index Machinery & Electricals_MAB_ELE_PRO276',
       'Germany_Shipments Index Machinery & Electricals_MAB_ELE_SHP276',
       'Italy_Production Index Machinery & Electricals_MAB_ELE_PRO380',
       'Italy_Shipments Index Machinery & Electricals_MAB_ELE_SHP380',
       'Japan_Production Index Machinery & Electricals_MAB_ELE_PRO392',
       'Japan_Shipments Index Machinery & Electricals_MAB_ELE_SHP392',
       'Switzerland_Production Index Machinery & Electricals_MAB_ELE_PRO756',
       'Switzerland_Shipments Index Machinery & Electricals_MAB_ELE_SHP756',
       'United Kingdom_Production Index Machinery & Electricals_MAB_ELE_PRO826',
       'United Kingdom_Shipments Index Mach

## materials_price dataframe

In [27]:
# Slice the dataframe and keep columns 18 to 24
materials_price = data.iloc[:, 18:25]

In [28]:
materials_price.columns

Index(['Europe_World: Price of Base Metals_RohiBASEMET1000_org',
       'Europe_World: Price of Energy_RohiENERGY1000_org',
       'Europe_World: Price of Metals  & Minerals_RohiMETMIN1000_org',
       'Europe_World: Price of Natural gas index_RohiNATGAS1000_org',
       'Europe_World: Price of Crude oil, average_RohCRUDE_PETRO1000_org',
       'Europe_World: Price of Copper_RohCOPPER1000_org',
       'Europe_United States: EUR in LCU_WKLWEUR840_org'],
      dtype='object')

## producer_price dataframe

In [31]:
# Slice the dataframe and keep columns from 25 to 30
producer_price = data.iloc[:, 25:31]

In [32]:
producer_price.columns

Index(['Producer Prices_United States: Electrical equipment_PRI27840_org',
       'Producer Prices_United Kingdom: Electrical equipment_PRI27826_org',
       'Producer Prices_Italy: Electrical equipment_PRI27380_org',
       'Producer Prices_France: Electrical equipment_PRI27250_org',
       'Producer Prices_Germany: Electrical equipment_PRI27276_org',
       'Producer Prices_China: Electrical equipment_PRI27156_org'],
      dtype='object')

## production_index dataframe

In [34]:
# Slice the dataframe and keep columns from 31 to the end
production_index = data.iloc[:, 31:]

In [35]:
production_index.columns

Index(['production index_United States: Machinery and equipment n.e.c._PRO28840_org',
       'production index_World: Machinery and equipment n.e.c._PRO281000_org',
       'production index_Switzerland: Machinery and equipment n.e.c._PRO28756_org',
       'production index_United Kingdom: Machinery and equipment n.e.c._PRO28826_org',
       'production index_Italy: Machinery and equipment n.e.c._PRO28380_org',
       'production index_Japan: Machinery and equipment n.e.c._PRO28392_org',
       'production index_France: Machinery and equipment n.e.c._PRO28250_org',
       'production index_Germany: Machinery and equipment n.e.c._PRO28276_org',
       'production index_United States: Electrical equipment_PRO27840_org',
       'production index_World: Electrical equipment_PRO271000_org',
       'production index_Switzerland: Electrical equipment_PRO27756_org',
       'production index_United Kingdom: Electrical equipment_PRO27826_org',
       'production index_Italy: Electrical equipment_